In [328]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [329]:
import zipfile

path_to_zip_file = '/content/drive/MyDrive/model_result.zip'
directory_to_extract_to = '/content/drive/MyDrive/dmp/result_folder'


with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [330]:
import os
import pandas as pd

file_list = os.listdir('/content/drive/MyDrive/dmp/fraction_ratio')

In [331]:
file_list

['menu2_df.csv',
 'menu1_df.csv',
 'login5_df.csv',
 'session3_df.csv',
 'session2_df.csv',
 'session1_df.csv',
 'menu4_df.csv',
 'login3_df.csv',
 'login2_df.csv',
 'login1_df.csv',
 'menu3_df.csv',
 'login4_df.csv',
 'info_df.csv',
 'menu2_df',
 'menu1_df',
 'login5_df',
 'menu4_df',
 'login3_df',
 'login2_df',
 'login1_df',
 'menu3_df',
 'login4_df',
 'info_df',
 'session3_df',
 'session2_df',
 'session1_df']

In [332]:
result = pd.read_csv('/content/drive/MyDrive/dmp/result_folder/model_result/login2_df.csv')
result

,Timestamp,LOGIN-02-Request,LOGIN-02-Fail,LOGIN-02-ratio,kmeans_anomaly,IF_anomaly
0,2018-01-01 00:05:00,2152.0,312.0,0.855019,1,1
1,2018-01-01 00:10:00,1628.0,184.0,0.886978,0,1
2,2018-01-01 00:15:00,1530.0,184.0,0.879739,0,1
3,2018-01-01 00:20:00,1475.0,187.0,0.873220,0,1
4,2018-01-01 00:25:00,1418.0,168.0,0.881523,0,1
...,...,...,...,...,...,...
105114,2018-12-31 23:35:00,4076.0,143.0,0.964917,1,1
105115,2018-12-31 23:40:00,4325.0,145.0,0.966474,1,1
105116,2018-12-31 23:45:00,4528.0,183.0,0.959585,1,1
105117,2018-12-31 23:50:00,5636.0,194.0,0.965578,1,1


In [333]:
results = pd.read_csv('/content/drive/MyDrive/dmp/dataset/data/Media/Media_INFO.csv')
results = results.set_index(results['Timestamp'])
indexs = results.index

In [334]:
indexs

Index(['20170101_0000-0005', '20170101_0005-0010', '20170101_0010-0015',
       '20170101_0015-0020', '20170101_0020-0025', '20170101_0025-0030',
       '20170101_0030-0035', '20170101_0035-0040', '20170101_0040-0045',
       '20170101_0045-0050',
       ...
       '20181231_2310-2315', '20181231_2315-2320', '20181231_2320-2325',
       '20181231_2325-2330', '20181231_2330-2335', '20181231_2335-2340',
       '20181231_2340-2345', '20181231_2345-2350', '20181231_2350-2355',
       '20181231_2355-0000'],
      dtype='object', name='Timestamp', length=210240)

In [335]:
import pandas as pd

def save_result(file):

  results = pd.read_csv('/content/drive/MyDrive/dmp/dataset/data/Media/Media_INFO.csv')
  results = results.set_index(results['Timestamp'])
  index_list = results.iloc[105120:,].index


  result = pd.read_csv(file)
  kmeans = pd.DataFrame()
  ifanom = pd.DataFrame()

  for i in result.columns:
    if 'kmeans_anomaly' in i:
      kmeans['Prediction'] = result[i]
      if kmeans.loc[105118,'Prediction'] == 0:
        kmeans['Prediction'] = kmeans['Prediction'].shift(periods=1)
        kmeans.loc[105119,'Prediction'] = 1
      elif kmeans.loc[105118,'Prediction'] == 1:
        kmeans['Prediction'] = kmeans['Prediction'].shift(periods=1)
        kmeans.loc[105119,'Prediction'] = 1

      kmeans['Prediction'] = kmeans['Prediction'].fillna(0)


    if 'IF_anomaly' in i:
      ifanom['Prediction'] = result[i]
      if ifanom.loc[105118,'Prediction'] == 0:
        ifanom['Prediction'] = ifanom['Prediction'].shift(periods=1)
        ifanom.loc[105119,'Prediction'] = 1
      elif ifanom.loc[105118,'Prediction'] == 1:
        ifanom['Prediction'] = ifanom['Prediction'].shift(periods=1)
        ifanom.loc[105119,'Prediction'] = 1

      ifanom['Prediction'] = ifanom['Prediction'].fillna(0)


  try:

    
    kmeans.to_csv(file[:-4]+'/k_result.csv',index=False)
    ifanom.to_csv(file[:-4]+'/if_result.csv',index=False)

  except:

    os.makedirs(file[:-4])
    kmeans.to_csv(file[:-4]+'/k_result.csv',index=False)
    ifanom.to_csv(file[:-4]+'/if_result.csv',index=False)


  return kmeans, ifanom

In [336]:
total_k_result = pd.DataFrame()
total_if_result = pd.DataFrame()

for i in file_list:
  
  if 'DS' in i:
    continue

  if 'csv' in i:

    a,b = save_result('/content/drive/MyDrive/dmp/fraction_ratio/'+i)
    a[i+'k_result'] = a['Prediction']
    b[i+'if_result'] = b['Prediction']
    
    total_k_result = pd.concat([total_k_result, a[i+'k_result']], axis = 1)
    total_if_result = pd.concat([total_if_result, b[i+'if_result']], axis = 1) 

total_k_result.to_csv('/content/drive/MyDrive/dmp/total_k_result1.csv',index=False)
total_if_result.to_csv('/content/drive/MyDrive/dmp/total_if_result1.csv',index=False)

In [352]:
total_k_result.sum()

menu2_df.csvk_result       210.0
menu1_df.csvk_result       210.0
login5_df.csvk_result      211.0
session3_df.csvk_result    212.0
session2_df.csvk_result    212.0
session1_df.csvk_result    225.0
menu4_df.csvk_result       210.0
login3_df.csvk_result      211.0
login2_df.csvk_result      211.0
login1_df.csvk_result      211.0
menu3_df.csvk_result       210.0
login4_df.csvk_result      211.0
info_df.csvk_result        210.0
dtype: float64

In [345]:
k = total_k_result.sum(axis=1)
k0 = k[k==0].index
media_answer = pd.DataFrame(index = total_k_result.index,columns = ['Prediction'])
media_answer.loc[k0,'Prediction'] =0

In [351]:
total_k_result

,menu2_df.csvk_result,menu1_df.csvk_result,login5_df.csvk_result,session3_df.csvk_result,session2_df.csvk_result,session1_df.csvk_result,menu4_df.csvk_result,login3_df.csvk_result,login2_df.csvk_result,login1_df.csvk_result,menu3_df.csvk_result,login4_df.csvk_result,info_df.csvk_result
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
105116,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
105117,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
105118,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0


In [346]:
k_media_answer = media_answer.fillna(1)
k_media_answer.to_csv('/content/drive/MyDrive/dmp/k_media_answer1.csv',index=False)

In [347]:
k_media_answer

,Prediction
0,0
1,0
2,0
3,0
4,0
...,...
105115,1
105116,1
105117,1
105118,1


In [339]:
k_media_answer.sum()

Prediction    916
dtype: int64

In [340]:
f = total_if_result.sum(axis=1)
f0 = f[f==0].index
media_f_answer = pd.DataFrame(index = total_if_result.index,columns = ['Prediction'])
media_f_answer.loc[f0,'Prediction'] =0

In [341]:
if_media_answer = media_f_answer.fillna(1)
if_media_answer.to_csv('/content/drive/MyDrive/dmp/if_media_answer1.csv',index=False)

In [349]:
if_media_answer.sum()

Prediction    1231
dtype: int64

In [343]:
if_media_answer

,Prediction
0,0
1,0
2,0
3,0
4,0
...,...
105115,1
105116,1
105117,1
105118,1
